# Predict movie offer suggestion using IBM Watson Machine Learning

This notebook introduces commands for getting data and for basic data cleaning and exploration, pipeline creation, model training, model persistance to Watson Machine Learning repository, model deployment, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.


## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model in Watson Machine Learning repository.
-  Deploy a model for online scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.



## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create spark ml model](#model)
4.	[Persist model](#save)
5.	[Predict locally and visualize](#predict)
6.	[Deploy and score in a Cloud](#deploy)


<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Upload **movie.csv** data as a data asset in IBM Data Science Experience.
-  Make sure that you are using a Spark 2.0 kernel.


<a id="load"></a>
## 2.  Load and explore data

IBM Data Science Experience (DSX) makes it easy to load your files with a few clicks!

**Action**: Import the data and add .option('inferSchema','true)

In [31]:

import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': '1d01a4b956544ae193baced39418dffc',
    'region': 'dallas',
    'user_id': '284703554b9c4314be591d5858cb32bb',
    'username': 'member_d42cb5e9f21e150269239deba3605b754a22cff0',
    'password': 'ao*9J2?vmvkO(HYe'
}

configuration_name = 'os_fd0d395ea9634e6ba8708dc2eebb3e12_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_3 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema','true')\
  .load(bmos.url('MachineLearningProject', 'movies.csv'))
df_data_3.take(5)


[Row(title=u'Rounders', GENDER=u'Female', SENIORCITIZEN=0, DEPENDENTS=1, TENURE=39, PAPERLESSBILLING=1, PAYMENTMETHOD=u'Credit card (automatic)', MONTHLYCHARGES=99.95),
 Row(title=u'Rounders', GENDER=u'Female', SENIORCITIZEN=0, DEPENDENTS=1, TENURE=71, PAPERLESSBILLING=0, PAYMENTMETHOD=u'Bank transfer (automatic)', MONTHLYCHARGES=20.1),
 Row(title=u'Rounders', GENDER=u'Female', SENIORCITIZEN=0, DEPENDENTS=1, TENURE=72, PAPERLESSBILLING=1, PAYMENTMETHOD=u'Electronic check', MONTHLYCHARGES=78.95),
 Row(title=u'Payback', GENDER=u'Male', SENIORCITIZEN=0, DEPENDENTS=0, TENURE=12, PAPERLESSBILLING=0, PAYMENTMETHOD=u'Bank transfer (automatic)', MONTHLYCHARGES=81.7),
 Row(title=u'Payback', GENDER=u'Male', SENIORCITIZEN=0, DEPENDENTS=0, TENURE=4, PAPERLESSBILLING=1, PAYMENTMETHOD=u'Electronic check', MONTHLYCHARGES=73.9)]

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  count all the records
-  print top five records

In [32]:
df = df_data_3

df.printSchema()
print "# of records: " + str(df.count())

root
 |-- title: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- SENIORCITIZEN: integer (nullable = true)
 |-- DEPENDENTS: integer (nullable = true)
 |-- TENURE: integer (nullable = true)
 |-- PAPERLESSBILLING: integer (nullable = true)
 |-- PAYMENTMETHOD: string (nullable = true)
 |-- MONTHLYCHARGES: double (nullable = true)

# of records: 1165


We can see that there are 1165 rows and we have 7 fields we will use to predict the title (label) of the movie.

In [33]:
df.show(5)

+--------+------+-------------+----------+------+----------------+--------------------+--------------+
|   title|GENDER|SENIORCITIZEN|DEPENDENTS|TENURE|PAPERLESSBILLING|       PAYMENTMETHOD|MONTHLYCHARGES|
+--------+------+-------------+----------+------+----------------+--------------------+--------------+
|Rounders|Female|            0|         1|    39|               1|Credit card (auto...|         99.95|
|Rounders|Female|            0|         1|    71|               0|Bank transfer (au...|          20.1|
|Rounders|Female|            0|         1|    72|               1|    Electronic check|         78.95|
| Payback|  Male|            0|         0|    12|               0|Bank transfer (au...|          81.7|
| Payback|  Male|            0|         0|     4|               1|    Electronic check|          73.9|
+--------+------+-------------+----------+------+----------------+--------------------+--------------+
only showing top 5 rows



Top 5 rows

<a id="model"></a>
## 3. Create an Apache Spark machine learning model

In this section we will prepare data, create an Apache Spark machine learning pipeline, and train a model.


### 3.1:  Prepare data

In this subsection we will split our data into: training, test, and predict datasets.

In [34]:
split_data = df.randomSplit([0.7, 0.28, 0.02], 24)

training_data = split_data[0]
test_data = split_data[1]
predict_data = split_data[2]

print "Training records: " + str(training_data.count())
print "Test records: " + str(test_data.count())
print "Prediction records: " + str(predict_data.count())

Training records: 808
Test records: 340
Prediction records: 17


As you can see our data has been successfully split into three datasets: 

-  The training dataset, which is the largest group, is used for training.
-  The test dataset will be used for model evaluation and is used to test the assumptions of the model.
-  The predict dataset will be used for prediction.

### 3.2:  Create pipeline and train a model

In this section we create an Apache Spark machine learning pipeline and then train the model.

First we need to import several packages that will be used in the next few steps.

In [35]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

First we need to convert all the string fields to numeric values.

In [36]:
stringIndexer_labels = StringIndexer(inputCol="title", outputCol="label").fit(df)
stringIndexer_gender = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX").fit(df)
stringIndexer_paymentmethod = StringIndexer(inputCol="PAYMENTMETHOD", outputCol="PAYMENTMETHOD_IX").fit(df)

Create a feature vector by combining all features together.

In [37]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX","SENIORCITIZEN","DEPENDENTS","TENURE","PAPERLESSBILLING",
                                                     "PAYMENTMETHOD_IX","MONTHLYCHARGES"], outputCol="features")

Next we define a Random Forest estimator.

In [38]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Next we convert the indexed labels back to the original label.

In [39]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_labels.labels)

Now we will put all the steps into a pipeline. 

In [40]:
pipeline_rf = Pipeline(stages=[stringIndexer_labels,stringIndexer_gender, stringIndexer_paymentmethod,
                               vectorAssembler_features, rf, labelConverter])

Now we will create a model using our pipeline and the training_data dataset.

In [41]:
model_rf = pipeline_rf.fit(training_data)

Now we will check our model accuracy using our test_data dataset.

In [42]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

Accuracy = 0.473529
Test Error = 0.526471


At this point we would tune the model for desired accuracy, for this example we will move on.

<a id="save"></a>
## 4. Persist model in IBM Watson Machine Learning

In this section you will learn how to store your pipeline and model in Watson Machine Learning repository by using python client libraries.

First, you must import client libraries.

**Note**: Apache Spark 2.0 or higher is required.

In [43]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Use your Watson Machine Learning service instance credentials below.



In [16]:
username = '<from your watson ml service>'
password = '<from your watson ml service>'
service_path = '<from your watson ml service>'
instance_id = '<from your watson ml service>'


**Tip**: service_path, user and password can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

In [17]:
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

Create model artifact (abstraction layer)

In [18]:
model_artifact = MLRepositoryArtifact(model_rf, training_data=training_data, name="Movie Prediction with Python")

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name. (It is this model name that is displayed by the Watson Machine Learning service).

### 4.1: Save pipeline and model

In [19]:
saved_model = ml_repository_client.models.save(model_artifact)

Get saved model metadata from Watson Machine Learning using the meta.available_props() method.

In [20]:
saved_model.meta.available_props()

['inputDataSchema',
 'evaluationMetrics',
 'pipelineVersionHref',
 'modelVersionHref',
 'trainingDataRef',
 'pipelineType',
 'creationTime',
 'lastUpdated',
 'label',
 'authorEmail',
 'trainingDataSchema',
 'authorName',
 'version',
 'modelType',
 'runtime',
 'evaluationMethod']

**Tip**:  **modelVersionHref** is our model unique id in Watson Machine Learning.

In [21]:
print saved_model.meta.prop("modelVersionHref")

https://ibm-watson-ml.mybluemix.net/v2/artifacts/models/44453df1-69c4-40cd-909c-bc7b5f3262a6/versions/c1acb55b-7223-4bd8-82b6-c408b6df6e3e


### 4.2: Load model

Now that we saved the model we will load it and verify the name.

In [22]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

In [23]:
print str(loadedModelArtifact.name)

Movie Prediction with Python


<a id="predict"></a>
## 5. Predict locally and visualize

In this section we will score test data using the loaded model.

### 5.1: Make local prediction using loaded model and predict data

In [24]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

In [25]:
predictions.show(3)

+----------+------+-------------+----------+------+----------------+--------------------+--------------+-----+---------+----------------+--------------------+--------------------+--------------------+----------+--------------+
|     title|GENDER|SENIORCITIZEN|DEPENDENTS|TENURE|PAPERLESSBILLING|       PAYMENTMETHOD|MONTHLYCHARGES|label|GENDER_IX|PAYMENTMETHOD_IX|            features|       rawPrediction|         probability|prediction|predictedLabel|
+----------+------+-------------+----------+------+----------------+--------------------+--------------+-----+---------+----------------+--------------------+--------------------+--------------------+----------+--------------+
|Inside Job|  Male|            0|         1|    28|               1|Bank transfer (au...|          74.9|  9.0|      0.0|             1.0|[0.0,0.0,1.0,28.0...|[0.11428571428571...|[0.00571428571428...|       9.0|    Inside Job|
|Inside Job|  Male|            0|         1|    53|               1|        Mailed check|   

In [26]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

+--------------------+-----+
|      predictedLabel|count|
+--------------------+-----+
|           Spotlight|    1|
|              Nomads|    2|
|             RoboCop|    2|
|    Punch-Drunk Love|    2|
|            Pit Stop|    1|
|          Rocket Men|    2|
|          Inside Job|    3|
|Prisoners of the Sun|    1|
|        Paid in Full|    3|
+--------------------+-----+



<a id="deploy"></a>
## 6. Deploy and create online scoring endpoint

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Swagger Documentation](http://watson-ml-api.mybluemix.net/).

To work with the Watson Machine Leraning REST API you must generate an access token. To do that you can use the following sample code:

In [27]:
import urllib3, requests, json

headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

Now that we have the token we can create an online scoring endpoint.

First we will check the model for existing deployments and get the deployments url, then we will create the online deployment.

In [28]:
published_model_details = service_path + "/v3/wml_instances/" + instance_id + "/published_models/"\
+ loadedModelArtifact.uid 
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

response_get_model_details = requests.get(published_model_details, headers=header)

print 'Existing deployment count: ' + str(json.loads(response_get_model_details.text).get('entity').get('deployments').get('count'))
deployments_endpoint = json.loads(response_get_model_details.text).get('entity').get('deployments').get('url')
print deployments_endpoint

Existing deployment count: 0
https://ibm-watson-ml.mybluemix.net/v3/wml_instances/40cda31c-7686-4b23-946c-bd2d5bf7fab3/published_models/44453df1-69c4-40cd-909c-bc7b5f3262a6/deployments


In [29]:
payload_online_endpoint = {"name": "Movie Prediction Deployment", "description": "Movie prediction endpoint\
for suggesting movies to customers.", "type": "online"}
response_online = requests.post(deployments_endpoint, json=payload_online_endpoint, headers=header)

scoring_endpoint = json.loads(response_online.text).get('entity').get('scoring_url')
print scoring_endpoint

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/40cda31c-7686-4b23-946c-bd2d5bf7fab3/published_models/44453df1-69c4-40cd-909c-bc7b5f3262a6/deployments/7d53c96f-c277-47d7-a186-3e8adc62ad97/online


Now we can send (POST) a new scoring request to our deployed model to get a movie prediction.

In [30]:
payload_scoring = {"fields": ["GENDER","SENIORCITIZEN","DEPENDENTS","TENURE","PAPERLESSBILLING",\
                             "PAYMENTMETHOD", "MONTHLYCHARGES"],"values": [["Male",0,0,25,1,"Credit card (automatic)",79.95]]}
response_scoring = requests.post(scoring_endpoint, json=payload_scoring, headers=header)

print json.loads(response_scoring.text)["values"][0]\
[len(json.loads(response_scoring.text)["values"][0])-1]


An American Werewolf in London


**Now we have a working online endpoint to use in our call center application**